# 交互式特征抽取

建立在文本向量化表示的基础上进行交互式的特征的抽取

# 使用scipy库中的距离计算库进行计算

# 特征抽取 
命名原则为 单独某个句子的特征使用q1或者q2开头

两个句子共同特征 命名方式统一为q12

使用 距离计算库scipy.spatial.distance 计算两个向量距离

对之前tfidf cos方法的补充

In [1]:
import pickle
import os
import re
import numpy
import numpy as np
import pandas as pd
import copy
import time
import scipy.spatial.distance as dist
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def read_pickle(fname):
    with open(fname,'rb') as file:
        return pickle.load(file)
def dump_pickle(obj,fname):
    with open(fname,'wb') as file:
        pickle.dump(obj,file)

In [3]:
merge_df = pd.read_pickle('../data/pure_data/merge_df_2018111172053.pkl')

In [4]:
all_df = pd.read_csv('../data/raw_data/question_id.csv')

In [5]:
feature_stop_columns = ['qid1','qid2','label','q1_w_list', 'q1_c_list', 'q2_w_list', 'q2_c_list']

In [6]:
def what_time_now():
    localtime = time.localtime()
    return '{0:0>4}{1:0>2}{2:0>2}{3:0>2}{4:0>2}'.format(localtime[0],localtime[1],localtime[2],localtime[3],localtime[4])

In [7]:
def interaction_output_distance(func,matrix1,matrix2):
    '''
    调用scipy函数计算matrix12对应行的距离
    matrix12 shape应当一致 为数据量 * 特征维度
    返回值为 数据量 * 1的向量 代表q1 和 q2的交互
    '''
    result = []
    for index,vec in enumerate(matrix1):
        result.append(func(vec.toarray(),matrix2[index].toarray()))
    return result

In [8]:
def scipy_dist_interaction_df_update(func,df,matrix1,matrix2,label='w_tfidf_1gram'):
    func_name = func.__name__
    new_label = 'q12_{0}_{1}'.format(label,func_name)
    df[new_label] = interaction_output_distance(func,matrix1,matrix2)
    # 平滑
    df = df.fillna(value=1e-6)
    # 归一化
    max_val = max(df[new_label])
    min_val = min(df[new_label])
    df[new_label] = df[new_label].apply(lambda x:(x-min_val)/(max_val-min_val))
    return df

In [9]:
def distance_pipline(df,matrix1,matrix2,label='w_tfidf_1gram'):
    distance_func_list = [dist.braycurtis,dist.canberra,dist.chebyshev,
                      dist.cityblock,dist.correlation,dist.dice,dist.euclidean,dist.cosine,dist.jaccard,dist.hamming,
                     dist.kulsinski,dist.sqeuclidean]
    for dist_fun in distance_func_list:
        df = scipy_dist_interaction_df_update(dist_fun,df,matrix1,matrix2,label)
    return df

In [10]:
blank_merge_df = merge_df[['qid1','qid2','label']].copy()

# 批量生成特征 TSNE单独做

In [11]:
def make_interaction(matrix_path,df):
    matrix_fname = matrix_path.split('/')[-1]
    matrix_granu = matrix_fname.split('_')[0]
    matrix_label = matrix_granu.split('.')[0]
    matrix_self = read_pickle(matrix_path)
    
    search_table = {}
    for i,_ in enumerate(all_df.itertuples()):
        search_table[all_df.loc[i,'qid']] = matrix_self[i]
    
    matrix1 = np.array([search_table[i] for i in df.qid1.tolist()])
    matrix2 = np.array([search_table[i] for i in df.qid2.tolist()])
    df = distance_pipline(df,matrix1,matrix2,label=matrix_label)
    return df

# 计算同时保存特征

In [12]:
base_origin_dir = '../data/feature/matrix/'
base_target_dir = '../data/feature/interaction/'

In [13]:
if not os.path.isdir(base_target_dir):
    os.mkdir(base_target_dir)

In [ ]:
counter = 0
method_list = ['onehot','tfidf']
for method in method_list:
    target_dir = '{}{}/'.format(base_target_dir,method)
    origin_dir = '{}{}/'.format(base_origin_dir,method)
    
    matrix_dir_list = [i for i in os.listdir(origin_dir) if i.split('.')[-1]!='txt']
    if not os.path.isdir(target_dir):
        os.mkdir(target_dir)

    for matrix_fname in matrix_dir_list:
        full_matrix_path = '{}{}'.format(origin_dir,matrix_fname)
        matrix_fname = '_'.join(matrix_fname.split('_')[:-1])
        inter_matrix_fname = '{}_interaction_df.pkl'.format(matrix_fname.split('.')[0])
        
        output_path = '{}{}'.format(target_dir,inter_matrix_fname)
#         print('Index:{} Fname:{}'.format(counter, full_matrix_path))
        print(output_path)
        if os.path.isfile(output_path) and os.path.isfile(csv_feature_path):
            continue
        temp_blank_merge_df = blank_merge_df.copy()

        temp_blank_merge_df = make_interaction(full_matrix_path,temp_blank_merge_df)
        temp_blank_merge_df.to_pickle(output_path)
        counter += 1

../data/feature/interaction/onehot/c_onehot_5gram_matrix_interaction_df.pkl


/home/lawbda/envs/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:1138: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()
